# Desafio 6

Neste desafio, vamos praticar _feature engineering_, um dos processos mais importantes e trabalhosos de ML. Utilizaremos o _data set_ [Countries of the world](https://www.kaggle.com/fernandol/countries-of-the-world), que contém dados sobre os 227 países do mundo com informações sobre tamanho da população, área, imigração e setores de produção.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [312]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
import scipy.stats as sct
from sklearn.datasets import load_digits, fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, Binarizer, KBinsDiscretizer,
    MinMaxScaler, StandardScaler, PolynomialFeatures
)

In [313]:
# Algumas configurações para o matplotlib.
#%matplotlib inline

from IPython.core.pylabtools import figsize


figsize(12, 8)

sns.set()

In [3]:
countries = pd.read_csv("countries.csv")

In [4]:
new_column_names = [
    "Country", "Region", "Population", "Area", "Pop_density", "Coastline_ratio",
    "Net_migration", "Infant_mortality", "GDP", "Literacy", "Phones_per_1000",
    "Arable", "Crops", "Other", "Climate", "Birthrate", "Deathrate", "Agriculture",
    "Industry", "Service"
]

countries.columns = new_column_names

#countries.head(5)

## Observações

Esse _data set_ ainda precisa de alguns ajustes iniciais. Primeiro, note que as variáveis numéricas estão usando vírgula como separador decimal e estão codificadas como strings. Corrija isso antes de continuar: transforme essas variáveis em numéricas adequadamente.

Além disso, as variáveis `Country` e `Region` possuem espaços a mais no começo e no final da string. Você pode utilizar o método `str.strip()` para remover esses espaços.

## Inicia sua análise a partir daqui

__Ajustando as variaveis numericas:__

In [11]:
#funçao para transformar os valores
def transforma_valores_df (df, nome_colunas):
    pd.set_option('mode.chained_assignment','warn')
    #salva tamanho da lista com o nome das colunas
    x = len(nome_colunas)
    #de i ate o tamanho da lista == 15
    for i in range(x):
        # salvando a qtd de elemntos da coluna em questao
        y = len(df[nome_colunas[i]])
        variavel = nome_colunas[i]
        #print(variavel)
        #pra cada elemento da lista transforma os valores
        #percorre valor por valor alterando a virgula por ponto 
        for j in range(y):
            valor = str(df.loc[j,variavel])
            if (valor != 'NaN') and (valor != 'nan') and (valor != 'Nan'):
                valor = float(valor.replace(",","."))
                #print(valor)
                df.loc[j,variavel] = valor
        #altera o tipo da coluna
        df[variavel] = df[variavel].astype(float)

In [12]:
#salvando em df auxiliar
aux = pd.DataFrame(countries)
lista_colunas_object = list(aux.select_dtypes(include=['object']))

In [13]:
#tirando os 2 primeiros elementos da lista ==  Region e Country
lista_colunas_object.pop(0)
lista_colunas_object.pop(0)

'Region'

In [14]:
#chamando a funcao para tranformar os valores
transforma_valores_df(aux,lista_colunas_object)

__Ajustando as variáveis Country e Region__

In [15]:
aux['Country'] = [x.strip() for x in aux['Country']]

In [16]:
aux['Region'] = [x.strip() for x in aux['Region']]

__Questao 2__

aux.isna().sum()

aux['Climate'].value_counts().nunique

aux.head(10)

In [267]:
#questao 2
#quantos países se encontram acima do 90º percentil?

discretizer = KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="quantile")

discretizer.fit(aux[["Pop_density"]])

pop_bins = discretizer.transform(aux[["Pop_density"]])

In [325]:
#discretizer.bin_edges_[0]

In [268]:
def get_interval(bin_idx, bin_edges):
  return f"{np.round(bin_edges[bin_idx], 2):.2f} ⊢ {np.round(bin_edges[bin_idx+1], 2):.2f}"

bin_edges_quantile = discretizer.bin_edges_[0]

print(f"Bins quantile")
print(f"interval: #elements\n")
for i in range(len(discretizer.bin_edges_[0])-1):
    print(f"{get_interval(i, bin_edges_quantile)}: {sum(pop_bins[:, 0] == i)}")

In [305]:
#print(get_interval(8, bin_edges_quantile))#8  pq vai de 0 a 9, entao 8 que é o nono intervalo

253.70 ⊢ 396.74


In [282]:
bin_edges_quantile = discretizer.bin_edges_[0]
pop_intervals = pd.Series(pop_bins.flatten().astype(np.int)).apply(get_interval, args=(bin_edges_quantile,))

In [330]:
#score_intervals

In [329]:
data_encoded = pd.concat([aux, pd.DataFrame(pop_intervals, columns=["Pop_density_interval"])], axis=1)

#data_encoded

In [312]:
ax2 = data_encoded[['Country', 'Pop_density_interval']]

In [324]:
qtd_paises = ax2[ax2['Pop_density_interval'] == '253.70 ⊢ 396.74']['Country'].count()

__Questao 3__

___Nessa questao eu errei da primeira vez pq eu tirei o nan da conta___

In [17]:
one_hot_encoder = OneHotEncoder(sparse=False, dtype=np.int, handle_unknown='ignore')

region_encoded = one_hot_encoder.fit_transform(aux[["Region"]])
#climate_encoded = one_hot_encoder.fit_transform(aux[["Climate"]])

region_encoded.shape

one_hot_encoder.categories_

region_encoded

In [21]:
X = np.array(aux[["Climate"]])

In [65]:
#tirando os valores nan de X
res = [] 
for i in X: 
    valor = str(i[0])
    if valor != 'nan': 
        #print(i[0])
        res.append(i[0]) 
        


In [66]:
#convertendo res para array dnv no msm formato de antes
X = np.array(res)
X = np.reshape(X, (-1,1)) 

In [68]:
enc = OneHotEncoder(sparse=False, dtype=np.int, handle_unknown='ignore')
climate_encoded = enc.fit_transform(X)

climate_encoded.shape

enc.categories_

climate_encoded

In [323]:
tota_novos_atributos = region_encoded.shape[1]+climate_encoded.shape[1]+1 
#+1 por causa do nan
#depois ver uma alternativa de fazer o procedimento com nan

tota_novos_atributos

__Questao 4__

___Essa questao nao entendi nem a pau ainda por que diabos tem que pegar e fazer um pipeline com padrozinacao___

In [93]:
aux_pipe = aux.copy()

__Pipeline__
- Preencha as variáveis do tipo int64 e float64 com suas respectivas medianas.
- Padronize essas variáveis.

In [94]:
#montando o pipeline
aux_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("zscore", StandardScaler())
])

In [95]:
#capturando as colunas especificadas
colunas_int_float = list(aux_pipe.select_dtypes(include=['int64','float64']))

In [96]:
#aplicando o pipeline
pipeline_transformation = aux_pipeline.fit_transform(aux_pipe[colunas_int_float])

In [98]:
#df depois do pipeline
axu_pos_pipeline = pd.DataFrame(pipeline_transformation, columns=colunas_int_float)

__Questao 5__

__Descubra o número de outliers da variável Net_migration segundo o método do _boxplot ___

__Nessa questao a primeira vez, eu errei porque entendi errado__

In [175]:
#aux.shape

In [325]:
net_migration_outlier = aux.Net_migration.copy()

net_migration_outlier

sns.boxplot(net_migration_outlier, orient="horizontal");

In [328]:
q1 = net_migration_outlier.quantile(0.25)
q3 = net_migration_outlier.quantile(0.75)
iqr = q3 - q1

non_outlier_interval_iqr = [q1 - 1.5 * iqr, q3 + 1.5 * iqr]

#print(f"Faixa considerada \"normal\": {non_outlier_interval_iqr}")

In [393]:
#encontrando os pontos fora da faixa
outliers_iqr = net_migration_outlier[(net_migration_outlier < non_outlier_interval_iqr[0]) | (net_migration_outlier > non_outlier_interval_iqr[1])]
outliers_iqr_abaixo = net_migration_outlier[(net_migration_outlier < non_outlier_interval_iqr[0])]
outliers_iqr_acima = net_migration_outlier[(net_migration_outlier > non_outlier_interval_iqr[1])]
outliers_abaixo = outliers_iqr_abaixo.count()
outliers_acima = outliers_iqr_acima.count()
removeria = False

print('Qtd de valores fora da faixa considerada normal', outliers_iqr.count())
print('Qtd total de valores da variavel', net_migration_outlier.count())
print('Qtd total de valores abaixo da faixa considerada normal', outliers_iqr_abaixo.count())
print('Qtd total de valores acima da faixa considerada normal', outliers_iqr_acima.count())

Você deveria remover da análise as observações consideradas outliers segundo esse método? Responda como uma tupla de três elementos (outliers_abaixo, outliers_acima, removeria?) ((int, int, bool)).

__Questoes 6 e 7__

__Nas questoes  6 e 7 errei a primeira vez de besteira tbm, porque noa contei certa a qtd de ocorrencia e sim alguma outra coisa que ainda nao descobri :|__

## Questão 1

Quais são as regiões (variável `Region`) presentes no _data set_? Retorne uma lista com as regiões únicas do _data set_ com os espaços à frente e atrás da string removidos (mas mantenha pontuação: ponto, hífen etc) e ordenadas em ordem alfabética.

In [6]:
def q1():
    #criando o grupo com as regioes
    grupo = aux.groupby(['Region'])
    #lista vazia
    lista_regioes = []
    #pegando tamanho do grupo
    y = len(grupo)
    # de x ate y == 11
    #adiciona na lista os nomes das regioes
    for x in range(y):
        #print(grupo['Region'].unique()[x][0])
        lista_regioes.append(grupo['Region'].unique()[x][0])
        
    return lista_regioes
    pass

## Questão 2

Discretizando a variável `Pop_density` em 10 intervalos com `KBinsDiscretizer`, seguindo o encode `ordinal` e estratégia `quantile`, quantos países se encontram acima do 90º percentil? Responda como um único escalar inteiro.

In [7]:
def q2():
    return int(qtd_paises)
    pass

# Questão 3

Se codificarmos as variáveis `Region` e `Climate` usando _one-hot encoding_, quantos novos atributos seriam criados? Responda como um único escalar.

In [9]:
def q3():
    return tota_novos_atributos
    pass

## Questão 4

Aplique o seguinte _pipeline_:

1. Preencha as variáveis do tipo `int64` e `float64` com suas respectivas medianas.
2. Padronize essas variáveis.

Após aplicado o _pipeline_ descrito acima aos dados (somente nas variáveis dos tipos especificados), aplique o mesmo _pipeline_ (ou `ColumnTransformer`) ao dado abaixo. Qual o valor da variável `Arable` após o _pipeline_? Responda como um único float arredondado para três casas decimais.

In [101]:
test_country = [
    'Test Country', 'NEAR EAST', -0.19032480757326514,
    -0.3232636124824411, -0.04421734470810142, -0.27528113360605316,
    0.13255850810281325, -0.8054845935643491, 1.0119784924248225,
    0.6189182532646624, 1.0074863283776458, 0.20239896852403538,
    -0.043678728558593366, -0.13929748680369286, 1.3163604645710438,
    -0.3699637766938669, -0.6149300604558857, -0.854369594993175,
    0.263445277972641, 0.5712416961268142
]

In [136]:
nome_colunas = list(aux_pipe.columns)

In [138]:
df_test_country = pd.DataFrame(np.array(test_country).reshape(1,20), columns=nome_colunas)

In [139]:
pipeline_transformation_test_country = aux_pipeline.fit_transform(df_test_country[colunas_int_float])

sem sentido total esse negocio de pipeline aki pra isso, pq parece q ja estao padronizados os dados e nem tem valor vazio...

In [11]:
def q4():
    return round(test_country[11], 3)
    pass

## Questão 5

Descubra o número de _outliers_ da variável `Net_migration` segundo o método do _boxplot_, ou seja, usando a lógica:

$$x \notin [Q1 - 1.5 \times \text{IQR}, Q3 + 1.5 \times \text{IQR}] \Rightarrow x \text{ é outlier}$$

que se encontram no grupo inferior e no grupo superior.

Você deveria remover da análise as observações consideradas _outliers_ segundo esse método? Responda como uma tupla de três elementos `(outliers_abaixo, outliers_acima, removeria?)` ((int, int, bool)).

In [12]:
def q5():
    # Retorne aqui o resultado da questão 4.
    return (int(outliers_abaixo), int(outliers_acima), removeria)
    pass

## Questão 6
Para as questões 6 e 7 utilize a biblioteca `fetch_20newsgroups` de datasets de test do `sklearn`

Considere carregar as seguintes categorias e o dataset `newsgroups`:

```
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
```


Aplique `CountVectorizer` ao _data set_ `newsgroups` e descubra o número de vezes que a palavra _phone_ aparece no corpus. Responda como um único escalar.

In [176]:
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']

In [177]:
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)

In [454]:
#essse é o tanto de documento..
#len(newsgroup.data)

In [395]:
##count_vect = CountVectorizer(stop_words= 'english', analyzer='word')
count_vect = CountVectorizer()
newsgroups_counts = count_vect.fit_transform(newsgroup.data)

In [433]:
words_idx = sorted([count_vect.vocabulary_.get(f'{word.lower()}') for word in [u"phone"]])

In [435]:
df_phone = pd.DataFrame(newsgroups_counts[:, words_idx].toarray(), columns=np.array(count_vect.get_feature_names())[words_idx])

In [445]:
#count_vect.vocabulary_['phone']

In [446]:
#print(count_vect.get_feature_names())

In [448]:
#count_vect.vocabulary_.get('phone')

In [ ]:
qtd_ocorrencias = df_phone['phone'].sum()

In [13]:
def q6():
    return int(qtd_ocorrencias)
    pass

## Questão 7

Aplique `TfidfVectorizer` ao _data set_ `newsgroups` e descubra o TF-IDF da palavra _phone_. Responda como um único escalar arredondado para três casas decimais.

In [449]:
tfidf_vectorizer = TfidfVectorizer()

tfidf_vectorizer.fit(newsgroup.data)

newsgroups_tfidf_vectorized = tfidf_vectorizer.transform(newsgroup.data)

In [451]:
df_phone_idf = pd.DataFrame(newsgroups_tfidf_vectorized[:, words_idx].toarray(), columns=np.array(count_vect.get_feature_names())[words_idx])

In [14]:
def q7():
    return float(round(df_phone_idf.sum(),3))
    pass